In [5]:
# 导入数据
# 对比四个简单的分类器
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# select k best features
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, matthews_corrcoef
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score

import warnings
warnings.filterwarnings('ignore')

In [48]:
# 读取数据
data_label = pd.read_excel("./feature/class.xlsx", index_col=1)
data_label = data_label[data_label['exclusion'] == 1]
# index去掉.npy
data_label.index = data_label.index.str.replace('.npy', '')

tongji_label = data_label[data_label['dataset'] == 'tongji']["class"]
xiangyang_label = data_label[data_label['dataset'] == 'xiangyang']["class"]
kits_label = data_label[data_label['dataset'] == 'kits']["class"]
henan_label = data_label[data_label['dataset'] == 'henan']["class"]

tongji_rad = pd.read_excel('./feature/rad/rad_tongji.xlsx', index_col=0).iloc[:, 37:]
xiangyang_rad = pd.read_excel('./feature/rad/rad_xiangyang.xlsx', index_col=0).iloc[:, 37:]
kits_rad = pd.read_excel('./feature/rad/rad_kits.xlsx', index_col=0).iloc[:, 37:]
henan_rad = pd.read_excel('./feature/rad/rad_henan.xlsx', index_col=0).iloc[:, 37:]

# 合并数据表格
feature_rad = pd.concat([tongji_rad, xiangyang_rad], axis=0)
label = pd.concat([tongji_label, xiangyang_label], axis=0)

# 按照index排序
feature_rad = feature_rad.sort_index()
label = label.sort_index()

kits_rad = kits_rad.sort_index()
kits_label = kits_label.sort_index()

henan_rad = henan_rad.sort_index()
henan_label = henan_label.sort_index()

assert all(feature_rad.index == label.index)
assert all(kits_rad.index == kits_label.index)
assert all(henan_rad.index == henan_label.index)

train_test_index = pd.read_excel("./index.xlsx", index_col=0)
train_index = train_test_index[train_test_index.index == 'train']['name'].values.ravel()
val_index = train_test_index[train_test_index.index == 'val']['name'].values.ravel()
X_train = feature_rad.loc[train_index]
y_train = label.loc[train_index]
X_val = feature_rad.loc[val_index]
y_val = label.loc[val_index]

# # 定义 X_train, y_train, X_val, y_val
# X_train, X_val, y_train, y_val = train_test_split(feature_rad, label, test_size=0.3, stratify=label, random_state=42)
# standardize the data
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_val = pd.DataFrame(scaler.transform(X_val), index=X_val.index, columns=X_val.columns)
X_henan = pd.DataFrame(scaler.transform(henan_rad), index=henan_rad.index, columns=henan_rad.columns)
X_kits = pd.DataFrame(scaler.transform(kits_rad), index=kits_rad.index, columns=kits_rad.columns)
y_henan = henan_label
y_kits = kits_label
assert all(X_kits.index == y_kits.index)
assert all(X_henan.index == y_henan.index)
y_train = y_train.values.ravel()
y_val = y_val.values.ravel()
y_henan = y_henan.values.ravel()
y_kits = y_kits.values.ravel()
# print(X_train.shape, y_train.shape)

In [33]:
def bootstrap_ci(metric_func, y_prob, y_true, n_iterations=1000, ci=0.95):
    """
    计算bootstrap置信区间
    """
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    size = len(y_true)
    scores = []
    
    rng = np.random.RandomState(42)  # 设置随机种子
    
    for _ in range(n_iterations):
        # 随机抽样（使用替换）
        indices = rng.randint(0, size, size=size)
        score = metric_func(y_true[indices], y_prob[indices])
        scores.append(score)
    
    # 计算置信区间
    sorted_scores = np.sort(scores)
    alpha = (1 - ci) / 2
    lower_bound = sorted_scores[int(alpha * n_iterations)]
    upper_bound = sorted_scores[int((1 - alpha) * n_iterations)]
    
    return (lower_bound, upper_bound)


# def bootstrap_ci(metric_func, y_prob, y_true, n_iterations=1000, ci=0.95):
#     """
#     使用bootstrap方法计算指标的置信区间
#     """
#     scores = []
#     size = len(y_true)
    
#     for _ in range(n_iterations):
#         # 随机抽样
#         indices = np.random.randint(0, size, size)
#         score = metric_func(y_true[indices], y_prob[indices])
#         scores.append(score)
    
#     # 计算置信区间
#     lower = np.percentile(scores, ((1-ci)/2)*100)
#     upper = np.percentile(scores, (1-(1-ci)/2)*100)
    
#     return round(lower, 3), round(upper, 3)

def calculate_metrics(y_prob, y_true):
    """
    计算不平衡二分类问题的评价指标，包括95%置信区间
    
    参数：
    y_prob: numpy array, 预测为正类(少数类)的概率值
    y_true: numpy array, 真实标签 (0为多数类，1为少数类)
    
    返回：
    dict: 包含评价指标和置信区间的字典
    """
    # 1. AUC
    auc_score = roc_auc_score(y_true, y_prob)
    auc_ci = bootstrap_ci(roc_auc_score, y_prob, y_true)
    
    # 2. 通过优化F1-score选择最佳阈值
    # precisions, recalls, thresholds = precision_recall_curve(y_true, y_prob)
    # f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    # f1_scores = np.nan_to_num(f1_scores)
    # best_threshold = thresholds[np.argmax(f1_scores[:-1])]
    best_threshold = 0.5
    
    # 使用最佳阈值进行预测
    y_pred = (y_prob >= best_threshold).astype(int)
    
    # 3. F1-score
    f1 = f1_score(y_true, y_pred, zero_division=0)
    f1_ci = bootstrap_ci(lambda y_t, y_p: f1_score(y_t, (y_p >= best_threshold).astype(int)),
                        y_prob, y_true)
    
    # 4. ACC
    acc = accuracy_score(y_true, y_pred)
    acc_ci = bootstrap_ci(lambda y_t, y_p: accuracy_score(y_t, (y_p >= best_threshold).astype(int)),
                         y_prob, y_true)
    
    # 5. MCC
    mcc = matthews_corrcoef(y_true, y_pred)
    mcc_ci = bootstrap_ci(lambda y_t, y_p: matthews_corrcoef(y_t, (y_p >= best_threshold).astype(int)),
                         y_prob, y_true)
    
    # 6. AUPRC
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    auprc = auc(recall, precision)
    
    def auprc_score(y_t, y_p):
        p, r, _ = precision_recall_curve(y_t, y_p)
        return auc(r, p)
    
    auprc_ci = bootstrap_ci(auprc_score, y_prob, y_true)
    
    # 7. Precision
    precisionscore = precision_score(y_true, y_pred)
    precision_ci = bootstrap_ci(lambda y_t, y_p: precision_score(y_t, (y_p >= best_threshold).astype(int)),
                              y_prob, y_true)

    # 8. Recall
    recallscore = recall_score(y_true, y_pred)
    recall_ci = bootstrap_ci(lambda y_t, y_p: recall_score(y_t, (y_p >= best_threshold).astype(int)),
                            y_prob, y_true)

    metrics = {
        'AUC': round(auc_score, 3),
        'AUC_CI': auc_ci,
        'F1': round(f1, 3),
        'F1_CI': f1_ci,
        'ACC': round(acc, 3),
        'ACC_CI': acc_ci,
        'MCC': round(mcc, 3),
        'MCC_CI': mcc_ci,
        'AUPRC': round(auprc, 3),
        'AUPRC_CI': auprc_ci,
        # 'threshold': round(best_threshold, 3),
        'Precision': round(precisionscore, 3),
        'Precision_CI': precision_ci,
        'Recall': round(recallscore, 3),
        'Recall_CI': recall_ci
    }
    return metrics

def format_metrics(metrics):
    """
    格式化指标输出，包含置信区间
    """
    formatted = {}
    for key in metrics:
        if key.endswith('_CI'):
            continue
        if key + '_CI' in metrics:
            formatted[key] = f"{metrics[key]} ({metrics[key+'_CI'][0]}-{metrics[key+'_CI'][1]})"
        else:
            formatted[key] = f"{metrics[key]}"
    
    return formatted

def merge_dicts_to_df(*dicts):
    """
    将多个评估指标字典合并为一个DataFrame
    
    参数:
    *dicts: 多个包含评估指标的字典
    
    返回:
    merged_df: 合并后的DataFrame
    """
    # 创建空的DataFrame保存所有结果
    merged_df = pd.DataFrame()
    
    # 创建索引列表
    new_index = ['train', 'inter_test', 'henan_test', 'kits_test']
    
    # 遍历所有字典
    for i, d in enumerate(dicts, 1):
        # 提取主要指标
        metrics = {
            'AUC': d['AUC'],
            'AUC_CI': f"({d['AUC_CI'][0]:.3f}-{d['AUC_CI'][1]:.3f})",
            'AUPRC': d['AUPRC'],
            'AUPRC_CI': f"({d['AUPRC_CI'][0]:.3f}-{d['AUPRC_CI'][1]:.3f})",
            'F1': d['F1'],
            'F1_CI': f"({d['F1_CI'][0]:.3f}-{d['F1_CI'][1]:.3f})",
            'ACC': d['ACC'],
            'ACC_CI': f"({d['ACC_CI'][0]:.3f}-{d['ACC_CI'][1]:.3f})",
            'MCC': d['MCC'],
            'MCC_CI': f"({d['MCC_CI'][0]:.3f}-{d['MCC_CI'][1]:.3f})",
            'Precision': d['Precision'],
            'Precision_CI': f"({d['Precision_CI'][0]:.3f}-{d['Precision_CI'][1]:.3f})",
            'Recall': d['Recall'],
            'Recall_CI': f"({d['Recall_CI'][0]:.3f}-{d['Recall_CI'][1]:.3f})",
        }
        
        # 转换为DataFrame并添加模型标识
        df = pd.DataFrame([metrics])
        
        # 合并到主DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # 重排列columns
    column_order = ['AUC', 'AUC_CI', 'AUPRC', 'AUPRC_CI', 
                    'F1', 'F1_CI', 'ACC', 'ACC_CI', 'MCC', 'MCC_CI',
                    'Precision', 'Precision_CI', 'Recall', 'Recall_CI']
    merged_df = merged_df[column_order]
    
    # 添加索引列
    merged_df.insert(0, 'Dataset', new_index)
    
    return merged_df

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

def create_classifiers() -> Dict:
    """创建分类器字典"""
    return {
        'logistic_regression': LogisticRegression(class_weight='balanced'),
        'decision_tree': DecisionTreeClassifier(class_weight='balanced'),
        'random_forest': RandomForestClassifier(class_weight='balanced'),
        'svm': SVC(class_weight='balanced', probability=True)
    }

def create_param_grids() -> Dict:
    """创建参数网格字典"""
    return {
        'logistic_regression': {
            'classifier__C': [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0],
            'classifier__penalty': ['l1', 'l2'],  # 使用l1正则化
            'classifier__max_iter': [2000],
            'classifier__tol': [1e-4],
        },
        'decision_tree': {
            'classifier__criterion': ['gini'],  # gini更常用
            'classifier__max_depth': [3, 4, 5, 6],
            'classifier__min_samples_split': [4, 5, 6, 7, 8],
            'classifier__min_samples_leaf': [1, 2, 3]
        },

        'random_forest': {
            'classifier__n_estimators': [90, 100, 110],         # 围绕100的核心取值
            'classifier__max_depth': [4, 5, 6],                 # 中等深度范围，避免过拟合
            'classifier__min_samples_split': [5, 6, 7],         # 适中的分裂条件
            'classifier__min_samples_leaf': [1, 2],             # 常用的叶节点限制
            'classifier__max_features': ['sqrt'],               # sqrt是最常用且效果好的选择
            'classifier__max_samples': [0.8, 0.9],              # 主流的采样比例
            'classifier__criterion': ['gini']                   # gini更常用且计算效率更高
        },
        
        'svm': {
            'classifier__C': [0.01, 0.05, 0.07, 0.1, 0.5, 1.0],  # 扩充正则化参数
            'classifier__kernel': ['linear', 'rbf', 'poly'],  # 添加核函数选项
            'classifier__gamma': ['scale', 'auto', 0.001, 0.01, 0.1]  # 扩充gamma选项
        }
    }

def evaluate_model(model, X: np.ndarray, y: np.ndarray, dataset_name: str) -> Optional[Dict]:
    """
    评估模型性能
    
    Parameters:
    -----------
    model : 训练好的模型
    X : array-like
        特征矩阵
    y : array-like
        标签向量
    dataset_name : str
        数据集名称
    
    Returns:
    --------
    Optional[Dict] : 包含评估指标的字典，如果评估失败则返回None
    """
    try:
        y_proba = model.predict_proba(X)[:, 1]
        metrics = calculate_metrics(y_proba, y)
        metrics['dataset'] = dataset_name
        return metrics
    except Exception as e:
        print(f"Error evaluating {dataset_name}: {str(e)}")
        return None

def run_grid_search(X_train: np.ndarray, y_train: np.ndarray, 
                   X_val: np.ndarray, y_val: np.ndarray, 
                   X_henan: np.ndarray, y_henan: np.ndarray, 
                   X_kits: np.ndarray, y_kits: np.ndarray, 
                   selected_classifiers: Optional[List[str]] = None, 
                   k_range: range = range(20, 50, 1)) -> pd.DataFrame:
    """
    运行网格搜索并评估模型
    
    Parameters:
    -----------
    X_train, y_train : 训练数据
    X_val, y_val : 验证数据
    X_henan, y_henan : 河南数据集
    X_kits, y_kits : KITS数据集
    selected_classifiers : 要评估的分类器列表
    k_range : 特征选择的k值范围
    
    Returns:
    --------
    pd.DataFrame : 评估结果表格
    """
    classifiers = create_classifiers()
    param_grids = create_param_grids()
    results = []
    
    if selected_classifiers:
        classifiers = {k: v for k, v in classifiers.items() if k in selected_classifiers}
    
    for k_num in k_range:
        for clf_name, clf in classifiers.items():
            print(f"\nProcessing k={k_num}, classifier={clf_name}")
            
            try:
                pipeline = Pipeline([
                    ('feature_selection', SelectKBest(mutual_info_classif, k=k_num)),
                    ('classifier', clf)
                ])
                
                grid_search = GridSearchCV(
                    pipeline,
                    param_grid=param_grids[clf_name],
                    cv=5,
                    # scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                    refit='roc_auc',
                    n_jobs=-1,
                    verbose=1
                )
                
                grid_search.fit(X_train, y_train)
                
                base_result = {
                    'k_num': k_num,
                    'classifier': clf_name,
                    'best_params': str(grid_search.best_params_),
                    'best_score': grid_search.best_score_
                }
                
                datasets = {
                    'validation': (X_val, y_val),
                    'henan': (X_henan, y_henan),
                    'kits': (X_kits, y_kits)
                }
                
                for dataset_name, (X, y) in datasets.items():
                    metrics = evaluate_model(grid_search.best_estimator_, X, y, dataset_name)
                    if metrics:
                        result = base_result.copy()
                        result.update(metrics)
                        results.append(result)
                
            except Exception as e:
                print(f"Error processing {clf_name} with k={k_num}: {str(e)}")
                continue
    
    results_df = pd.DataFrame(results)
    
    results_df.to_excel('model_evaluation_results_rad.xlsx', index=False)
    print("\nResults saved to 'model_evaluation_results_rad.xlsx'")
    
    return results_df

# 使用示例
results_df = run_grid_search(X_train, y_train, X_val, y_val, X_henan, y_henan, X_kits, y_kits)


Processing k=20, classifier=logistic_regression
Fitting 5 folds for each of 14 candidates, totalling 70 fits



Processing k=20, classifier=decision_tree
Fitting 5 folds for each of 60 candidates, totalling 300 fits

Processing k=20, classifier=random_forest
Fitting 5 folds for each of 108 candidates, totalling 540 fits

Processing k=20, classifier=svm
Fitting 5 folds for each of 90 candidates, totalling 450 fits

Processing k=21, classifier=logistic_regression
Fitting 5 folds for each of 14 candidates, totalling 70 fits

Processing k=21, classifier=decision_tree
Fitting 5 folds for each of 60 candidates, totalling 300 fits

Processing k=21, classifier=random_forest
Fitting 5 folds for each of 108 candidates, totalling 540 fits

Processing k=21, classifier=svm
Fitting 5 folds for each of 90 candidates, totalling 450 fits

Processing k=22, classifier=logistic_regression
Fitting 5 folds for each of 14 candidates, totalling 70 fits

Processing k=22, classifier=decision_tree
Fitting 5 folds for each of 60 candidates, totalling 300 fits

Processing k=22, classifier=random_forest
Fitting 5 folds for 

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

classifier = RandomForestClassifier(
n_estimators=110,
max_depth=6,
min_samples_split=6,
min_samples_leaf=1,
max_features='sqrt',
max_samples=0.8,
criterion='gini',
random_state=42,  # 添加随机种子以确保可重复性
n_jobs=-1,  # 使用所有CPU核心
class_weight='balanced'
)
# classifier = SVC(
# C=0.1,
# kernel='poly',
# gamma=0.01,
# probability=True,
# class_weight='balanced',
# random_state=42
# )

# 2. 构建pipeline
pipeline = Pipeline([
    ('feature_selection', SelectKBest(mutual_info_classif, k=43)),
    ('classifier', classifier)
])

# 3. 训练模型
pipeline.fit(X_train, y_train)

# 4. 在训练集和内部验证集上评估
train_proba = pipeline.predict_proba(X_train)[:, 1]
val_proba = pipeline.predict_proba(X_val)[:, 1]

# 5. 在两个测试集上评估
henan_proba = pipeline.predict_proba(X_henan)[:, 1]
kits_proba = pipeline.predict_proba(X_kits)[:, 1]

# 将计算指标转为表格
train_df = pd.DataFrame({
    'dataset': 'train',
    'name': X_train.index,
    'ground_truth': y_train,
    'probability': train_proba
})

val_df = pd.DataFrame({
    'dataset': 'val', 
    'name': X_val.index,
    'ground_truth': y_val,
    'probability': val_proba
})

henan_df = pd.DataFrame({
    'dataset': 'henan',
    'name': X_henan.index,
    'ground_truth': y_henan,
    'probability': henan_proba
})

kits_df = pd.DataFrame({
    'dataset': 'kits',
    'name': X_kits.index,
    'ground_truth': y_kits,
    'probability': kits_proba
})
# 合并所有DataFrame
final_df = pd.concat([train_df, val_df, henan_df, kits_df], ignore_index=True)
# 保存为Excel文件
final_df.to_excel('rad.xlsx', index=False)

# 6. 计算各项指标
result_train = calculate_metrics(train_proba, y_train)
result_val = calculate_metrics(val_proba, y_val)
result_henan = calculate_metrics(henan_proba, y_henan)
result_kits = calculate_metrics(kits_proba, y_kits)
print(result_train)
print(result_val)
print(result_henan)
print(result_kits)

# 合并字典
merged_results = merge_dicts_to_df(result_train, result_val, result_henan, result_kits)

# 保存为CSV文件
merged_results.to_excel('results_rad.xlsx', index=False)

{'AUC': 0.995, 'AUC_CI': (0.9906565656565656, 0.9983706720977598), 'F1': 0.832, 'F1_CI': (0.75, 0.9037037037037037), 'ACC': 0.966, 'ACC_CI': (0.9491833030852994, 0.9818511796733213), 'MCC': 0.815, 'MCC_CI': (0.7320750486967602, 0.8938776287820568), 'AUPRC': 0.957, 'AUPRC_CI': (0.9178303862596339, 0.9859161674451565), 'Precision': 0.887, 'Precision_CI': (0.8035714285714286, 0.9642857142857143), 'Recall': 0.783, 'Recall_CI': (0.6764705882352942, 0.8909090909090909)}
{'AUC': 0.787, 'AUC_CI': (0.7020043845912934, 0.8591934381408066), 'F1': 0.205, 'F1_CI': (0.04761904761904763, 0.375), 'ACC': 0.869, 'ACC_CI': (0.8227848101265823, 0.9113924050632911), 'MCC': 0.153, 'MCC_CI': (-0.023433127818198694, 0.3359656089406227), 'AUPRC': 0.325, 'AUPRC_CI': (0.17404024100779836, 0.4901531212407679), 'Precision': 0.308, 'Precision_CI': (0.07692307692307693, 0.5882352941176471), 'Recall': 0.154, 'Recall_CI': (0.03333333333333333, 0.3)}
{'AUC': 0.641, 'AUC_CI': (0.5377736663583103, 0.7456395348837209), 'F

In [113]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# 创建完整的管道
def create_training_pipeline():
    # 定义基础分类器
    base_classifier = RandomForestClassifier(
        n_estimators=110,
        max_depth=6,
        min_samples_split=6,
        min_samples_leaf=1,
        max_features='sqrt',
        max_samples=0.8,
        criterion='gini',
        random_state=42,  # 添加随机种子以确保可重复性
        n_jobs=-1,  # 使用所有CPU核心
        class_weight='balanced')

    # 创建完整的预处理和训练管道
    pipeline = Pipeline([
        ('feature_selection', SelectKBest(mutual_info_classif, k=4)),
        # ('smote', SMOTE(sampling_strategy='auto',random_state=42)),
        ('undersampler', RandomUnderSampler(sampling_strategy='auto', random_state=42)),
        ('classifier', base_classifier)
    ])
    
    return pipeline

# 创建管道
pipeline = create_training_pipeline()

# 训练模型
pipeline.fit(X_train, y_train)

# 获取预测概率
train_proba = pipeline.predict_proba(X_train)[:, 1]
val_proba = pipeline.predict_proba(X_val)[:, 1]
henan_proba = pipeline.predict_proba(X_henan)[:, 1]
kits_proba = pipeline.predict_proba(X_kits)[:, 1]

# 计算各项指标
result_train = calculate_metrics(train_proba, y_train)
result_val = calculate_metrics(val_proba, y_val)
result_henan = calculate_metrics(henan_proba, y_henan)
result_kits = calculate_metrics(kits_proba, y_kits)
print(result_train)
print(result_val)
print(result_henan)
print(result_kits)

# 合并字典
merged_results = merge_dicts_to_df(result_train, result_val, result_henan, result_kits)

# 保存为CSV文件
merged_results.to_excel('results_rad.xlsx', index=False)

{'AUC': 0.899, 'AUC_CI': (0.862, 0.935), 'F1': 0.376, 'F1_CI': (0.312, 0.443), 'ACC': 0.644, 'ACC_CI': (0.606, 0.684), 'MCC': 0.353, 'MCC_CI': (0.296, 0.408), 'AUPRC': 0.615, 'AUPRC_CI': (0.488, 0.731), 'Precision': 0.234, 'Precision_CI': (0.183, 0.288), 'Recall': 0.952, 'Recall_CI': (0.892, 1.0)}
{'AUC': 0.75, 'AUC_CI': (0.648, 0.839), 'F1': 0.261, 'F1_CI': (0.167, 0.359), 'ACC': 0.57, 'ACC_CI': (0.506, 0.633), 'MCC': 0.181, 'MCC_CI': (0.064, 0.286), 'AUPRC': 0.257, 'AUPRC_CI': (0.142, 0.45), 'Precision': 0.158, 'Precision_CI': (0.096, 0.225), 'Recall': 0.75, 'Recall_CI': (0.565, 0.92)}
{'AUC': 0.564, 'AUC_CI': (0.471, 0.659), 'F1': 0.324, 'F1_CI': (0.222, 0.429), 'ACC': 0.508, 'ACC_CI': (0.439, 0.578), 'MCC': 0.016, 'MCC_CI': (-0.127, 0.157), 'AUPRC': 0.305, 'AUPRC_CI': (0.19, 0.432), 'Precision': 0.237, 'Precision_CI': (0.152, 0.33), 'Recall': 0.512, 'Recall_CI': (0.37, 0.667)}
{'AUC': 0.68, 'AUC_CI': (0.558, 0.789), 'F1': 0.404, 'F1_CI': (0.283, 0.509), 'ACC': 0.593, 'ACC_CI': (0.5

In [54]:
import os
import shap
def save_feature_importance_plots(pipeline, X_train, y_train, X_val, y_val, feature_names, save_dir='feature_importance_plots'):
    """
    保存所有特征重要性相关的图为PDF
    """
    # 创建保存目录
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    # 1. 特征选择分析图
    selector = pipeline.named_steps['feature_selection']
    selected_mask = selector.get_support()
    selected_features = feature_names[selected_mask]
    scores = selector.scores_
    
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(selected_features)), 
            pd.DataFrame({'Feature': feature_names, 'Selected': selected_mask, 
                         'Mutual_Info_Score': scores})[selected_mask]['Mutual_Info_Score'])
    plt.xticks(range(len(selected_features)), selected_features, rotation=45, ha='right')
    plt.title('Mutual Information Scores of Selected Features')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'mutual_info_scores.pdf'))
    plt.close()
    
    # 2. Random Forest特征重要性图
    rf_model = pipeline.named_steps['classifier']
    importances = rf_model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)
    
    feature_importance = pd.DataFrame({
        'Feature': selected_features,
        'Importance': importances,
        'Std': std
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(12, 6))
    feature_importance.plot(kind='bar', x='Feature', y='Importance', yerr='Std',
                          capsize=5, alpha=0.8)
    plt.title('Random Forest Feature Importance')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'rf_importance.pdf'))
    plt.close()
    
    # 3. SHAP值分析图
    if isinstance(X_train, pd.DataFrame):
        X_selected = X_train.iloc[:, selected_mask].values
    else:
        X_selected = X_train[:, selected_mask]
    
    explainer = shap.TreeExplainer(rf_model)
    shap_values = explainer.shap_values(X_selected)
    
    if isinstance(shap_values, list):
        shap_values = shap_values[1]
    
    # SHAP summary plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_selected, 
                     feature_names=selected_features.tolist(),
                     show=False)
    plt.title('SHAP Summary Plot')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'shap_summary.pdf'))
    plt.close()
    
    # SHAP依赖图（top 3特征）
    mean_abs_shap = np.abs(shap_values).mean(0)
    top_features_idx = np.argsort(mean_abs_shap)[-3:]
    
    for idx in top_features_idx:
        plt.figure(figsize=(8, 6))
        shap.dependence_plot(idx, shap_values, X_selected,
                           feature_names=selected_features.tolist(),
                           show=False)
        plt.title(f'SHAP Dependence Plot - {selected_features[idx]}')
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f'shap_dependence_{selected_features[idx]}.pdf'))
        plt.close()
        
    # 4. 合并所有重要性指标并保存
    final_importance = pd.DataFrame({
        'Feature': selected_features,
        'Mutual_Info_Score': scores[selected_mask],
        'RF_Importance': importances,
        'RF_Importance_Std': std,
        'Mean_Abs_SHAP': mean_abs_shap
    })
    
    final_importance.to_csv(os.path.join(save_dir, 'feature_importance_metrics.csv'), index=False)
    
    print(f"所有图像和数据已保存到目录: {save_dir}")
    return final_importance

# 使用示例:
results = save_feature_importance_plots(
    pipeline=pipeline,
    X_train=X_train, 
    y_train=y_train,
    X_val=X_val, 
    y_val=y_val,
    feature_names=np.array(X_train.columns),
    save_dir='feature_importance_results'
)

所有图像和数据已保存到目录: feature_importance_results


<Figure size 1200x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>